In [ ]:
!pip install mlflow torchmetrics torchinfo --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=01d13b7d9e65efeef84e4e12a40f3a70d415ad01eb0ce5b3bd3a1ebce545ebd6
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [ ]:
import os
import shutil
import pprint

from random import random, randint
from mlflow import log_metric, log_param, log_artifacts
from mlflow.tracking import MlflowClient
import warnings

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchmetrics import Accuracy, F1Score
from torchvision import datasets
from torchvision.transforms import ToTensor

import mlflow
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch.optim as optim
import time
from sklearn.metrics import f1_score
import torch.nn.functional as F
#import tensorflow as tf
import GPUtil
from collections import deque

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.21.3


In [ ]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.21.3


In [ ]:
folder_path = '/content/drive/MyDrive/ap_proj1'
files_in_folder = os.listdir(folder_path + "/data")
print(files_in_folder)
print(os.getcwd())

['train_labels.csv', 'Test', 'Train', 'TrainCropped']
/content


In [ ]:
labels = pd.read_csv(folder_path + "/data/train_labels.csv")
image_folder = folder_path + "/data/Train/"
label_encoder = LabelEncoder()
labels["label"] = label_encoder.fit_transform(labels["label"])
num_classes = len(label_encoder.classes_)
print(num_classes)

9


In [ ]:
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///drive/MyDrive/ap_proj1/mlflowdata/mlruns.db --port 5000 &")# run tracking UI in the background
local_registry = "sqlite:///drive/MyDrive/ap_proj1/mlflowdata/mlruns.db"
model_name="CNN"
mlflow.set_tracking_uri(local_registry)
mlflow.set_experiment("/task2")

<Experiment: artifact_location='/content/mlruns/3', creation_time=1744459292020, experiment_id='3', last_update_time=1744459292020, lifecycle_stage='active', name='/task2', tags={}>

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
NGROK_AUTH_TOKEN = "2uryxEZ5QrQQ0SGn8gTWLAAT1ei_7Njezw5fQYwuwdb4mfhpk" # <- Paste your token here
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
public_url = ngrok.connect("5000")
print("MLflow Tracking UI:", public_url)

MLflow Tracking UI: NgrokTunnel: "https://ec94-35-197-92-10.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
class PokemonDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        img_name = os.path.join(self.img_dir, str(self.df.iloc[idx, 0]))
        if not img_name.endswith('.png'):
            img_name += ".png"
            image = Image.open(img_name)
        else:
            image = Image.open(img_name)

        if self.transform:
            image = self.transform(Image.open(img_name))

        if len(self.df.columns) > 1:  # Train Set has labels, Test does not.
            label = self.df.iloc[idx, 1]
            return image, label
        else:
            return image, -1  # X dont care for Test

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes, dropout=0.3):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)

        # At this point: 64x64 → 32x32 → 16x16 → 8x8 → 4x4
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(256 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
# Define transformations for better generalization
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(), # Random flipping left-right
    #transforms.RandomRotation(15), # Rotate randomly within ±15 degrees
    #transforms.ColorJitter(brightness=0.2, contrast=0.2), # Adjust brightness/contrast
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.0], std=[0.5, 0.5, 0.5, 1.0])
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten
])

In [ ]:
# Split the dataset into 3 (70% train, 10% val, 20% test)
dataset = PokemonDataset(labels, image_folder, transform=transform)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
batch_size = 16
n_epochs = 80
learning_rate = 0.001

In [ ]:
model = CNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pip_requirements = ["torch==2.6.0+cu124"]
model.to(device)

CNN(
  (conv1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def log_gpu_usage():
    gpus = GPUtil.getGPUs()
    for i, gpu in enumerate(gpus):
        mlflow.log_metric(f"gpu_{i}_memory_used_MB", gpu.memoryUsed)
        mlflow.log_metric(f"gpu_{i}_memory_total_MB", gpu.memoryTotal)
        mlflow.log_metric(f"gpu_{i}_utilization_percent", gpu.load * 100)


In [ ]:
# Function to check stopping condition
def early_stop(queue):
    if len(queue) < 5:
        return False  # Not enough data yet

    scores = np.array(queue)
    max_score = np.max(scores)
    min_score = np.min(scores)

    # Relative difference
    if max_score == 0:
        return False  # avoid division by zero

    relative_change = (max_score - min_score) / max_score

    return relative_change < 0.02  # less than 2% change

In [ ]:
# Training Loop
def training_loop():
    f1_queue = deque(maxlen=5)
    start_time = time.time()

    # Training loop
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
          images, labels = images.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

          log_gpu_usage()

        # Validation loop
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
          for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)  # Get predicted class indices
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

        # Calculate F1 Score
        f1 = f1_score(all_labels, all_preds, average="macro")
        f1_queue.append(f1)
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {running_loss/len(train_loader):.4f}, F1 Score: {f1:.4f}")
        mlflow.log_metric("loss", f"{running_loss/len(train_loader):.4f}")
        mlflow.log_metric("f1_score", f"{f1:.4f}")
        if early_stop(f1_queue):
            print("Early stopping triggered: F1 score change < 2% over last 5 epochs.")
            break

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")

    # Test loop
    model.eval()
    correct, total = 0, 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_preds, average="macro")
    mlflow.log_metric("eval_loss", f"{running_loss/len(train_loader):.4f}")
    mlflow.log_metric("eval_accuracy", f"{f1:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test F1 Score: {f1:.4f}")

In [ ]:
with mlflow.start_run() as run:
  params = {
        "epochs": n_epochs,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "loss_function": criterion.__class__.__name__,
        "metric_function": f1_score.__name__,
        "optimizer": "Adam"
        #"step_size": step_size,
        #"layer1_size": l1_size,
        #"layer2_size": l2_size,
        #"dropout": dropout,
        #"gamma": gamma
    }
  output_folder = os.path.join(folder_path, "outputs")
  if not os.path.exists(output_folder):
      os.makedirs(output_folder)

  # Log training parameters.
  mlflow.log_params(params)

  # Log model summary.
  with open(os.path.join(output_folder, "model_summary.txt"), "w") as f:
      f.write(str(summary(model)))

  log_artifacts(output_folder)
  mlflow.log_artifact(os.path.join(output_folder, "model_summary.txt"))

  training_loop()

  mlflow.pytorch.log_model(model, "model", pip_requirements=pip_requirements)#, input_example=sample_image.numpy())

Epoch [1/80], Loss: 2.0462, F1 Score: 0.2486
Epoch [2/80], Loss: 1.7629, F1 Score: 0.2588
Epoch [3/80], Loss: 1.6564, F1 Score: 0.3271
Epoch [4/80], Loss: 1.4790, F1 Score: 0.3636
Epoch [5/80], Loss: 1.2439, F1 Score: 0.3562
Epoch [6/80], Loss: 1.1346, F1 Score: 0.5768
Epoch [7/80], Loss: 1.0018, F1 Score: 0.3458
Epoch [8/80], Loss: 0.8941, F1 Score: 0.4440
Epoch [9/80], Loss: 0.8302, F1 Score: 0.6726
Epoch [10/80], Loss: 0.7216, F1 Score: 0.5907
Epoch [11/80], Loss: 0.6265, F1 Score: 0.7052
Epoch [12/80], Loss: 0.5035, F1 Score: 0.7712
Epoch [13/80], Loss: 0.4790, F1 Score: 0.5319
Epoch [14/80], Loss: 0.4386, F1 Score: 0.6701
Epoch [15/80], Loss: 0.3158, F1 Score: 0.8730
Epoch [16/80], Loss: 0.2885, F1 Score: 0.4780
Epoch [17/80], Loss: 0.2955, F1 Score: 0.6166
Epoch [18/80], Loss: 0.2356, F1 Score: 0.8996
Epoch [19/80], Loss: 0.1634, F1 Score: 0.9129
Epoch [20/80], Loss: 0.1943, F1 Score: 0.9747
Epoch [21/80], Loss: 0.1864, F1 Score: 0.6219
Epoch [22/80], Loss: 0.1564, F1 Score: 0.77

2025/04/14 22:11:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
client = MlflowClient()

print("List of all registered models")
print("=" * 80)
[print(pprint.pprint(dict(rm), indent=4)) for rm in client.search_registered_models()]

# Get a list of specific versions of the named models
print(f"\nList of Model = {model_name} and Versions")
print("=" * 80)
[pprint.pprint(dict(mv), indent=4) for mv in client.search_model_versions()]

List of all registered models
{   'aliases': {},
    'creation_timestamp': 1743092803691,
    'description': None,
    'last_updated_timestamp': 1743092803751,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1743092803751, current_stage='None', description=None, last_updated_timestamp=1743092803751, name='MLP', run_id='b2e8bf02d84647068e8b6324abad84bf', run_link=None, source='/content/mlruns/1/b2e8bf02d84647068e8b6324abad84bf/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>],
    'name': 'MLP',
    'tags': {}}
None

List of Model = CNN and Versions
{   'aliases': [],
    'creation_timestamp': 1743092803751,
    'current_stage': 'None',
    'description': None,
    'last_updated_timestamp': 1743092803751,
    'name': 'MLP',
    'run_id': 'b2e8bf02d84647068e8b6324abad84bf',
    'run_link': None,
    'source': '/content/mlruns/1/b2e8bf02d84647068e8b6324abad84bf/artifacts/sklearn-model',
    'status': 'READY',
    'st

[None]